In [4]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [5]:
# Reading in necessary files
cpi_by_region = pd.read_csv("../data/cpi_by_region.csv")
income = pd.read_excel("../data/median_income.xlsx", skiprows = 7 )
all_cpi = pd.read_excel("../data/cpi_data.xlsx", skiprows = 11)

c:\Users\kidsa\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Removing metadata
income_updated = income[1:]
income_updated = income_updated[:58]

In [7]:
# Converting year column into int dtype
income_updated['Race, Hispanic origin, and year'] = income_updated['Race, Hispanic origin, and year'].astype(str).str[:4]
income_updated['Race, Hispanic origin, and year'] = income_updated['Race, Hispanic origin, and year'].astype(int)

In [8]:
# Subsetting dataframe for necessary columns
income_updated = income_updated[['Race, Hispanic origin, and year','Median income']]

In [9]:
# Renaming columns
income_updated = income_updated.rename(columns = {'Race, Hispanic origin, and year':'year',"Median income":'median_income'})

In [8]:
# Exporting clean income data to data folder
income_updated.to_csv("../data/income_updated.csv")

In [10]:
# Turning monthly CPI data in yearly data
cpi_by_year = cpi_by_region.query("Region == 'U.S. city average' & Month == 'December'")

In [11]:
# Removing metadata and getting year to year percent change for CPI
all_cpi_data = all_cpi.sort_values(ascending = False, by = "Year")[1:]
all_cpi_data = all_cpi_data[['Year','Dec']]
all_cpi_data['yearly_change'] = (all_cpi_data['Dec'].pct_change(periods = -1))*100

In [12]:
# Renaming columns
all_cpi_data = all_cpi_data.rename(columns = {'Year':'year','Dec':'CPI','yearly_change':'yearly_change_c'})

In [13]:
# Shows the year to year percentage change for income
income_updated['yearly_change'] = (income_updated['median_income'].pct_change(periods = -1))*100
income_updated = income_updated.rename(columns = {"yearly_change":"yearly_change_i"})

In [14]:
# Merge cpi data with income data so we can calculate a column that shows how the value of our money has changed
cpi_income = all_cpi_data.merge(right = income_updated, how = 'inner', on = 'year')

In [15]:
# Create a column to show how the value of money changed each year
cpi_income['income_change'] = cpi_income['yearly_change_i'] - cpi_income['yearly_change_c']

# Sort values by year then run cumulative sum function
cpi_income = cpi_income.sort_values(by = 'year', ascending = True)
cpi_income['cum_income_change'] = cpi_income['income_change'].cumsum()

# Run cmulative sum function on CPI
cpi_income['cum_cpi_change'] = cpi_income['yearly_change_c'].cumsum()

In [20]:
# Export dataframes as csvs
all_cpi_data.to_csv("../data/all_cpi_updated.csv")
income_updated.to_csv("../data/accurate_income.csv")
cpi_income.to_csv("../data/cpi_income.csv")